In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

In [10]:
dataset = pd.read_csv("./data/friofrio/Train_TP2_Datos_2020-2C.csv", 
            parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                           'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                           'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

dataset2 = pd.read_csv("./data/friofrio/Test_TP2_Datos_2020-2C.csv", 
                       parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                                      'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                                      'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
#dataset = dataset.drop(columns= [Brand','Size','Product_Type'])
dataset2 = dataset2.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

In [11]:
#dataset.columns.intersection(dataset2.columns)
#dataset2.columns.intersection(['Region', 'Territory','Bureaucratic_Code', 'Billing_Country',
#                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
#               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter'])

dataset2.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

In [12]:
#limpio los datos no finalzizados
dataset = dataset.loc[(dataset['Stage']== 'Closed Won') | (dataset['Stage']== 'Closed Lost')]
dataset['Stage'].value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

In [13]:
#mergeo ambos DS con countes y ganados, en el caso de dataset 2 los cunters y los ganados son siempre 0 así no afectan los promedios.
dataset['ganado']= (dataset['Stage']== 'Closed Won' )
dataset.drop(columns= ['Stage'])
dataset['counter']= 1

dataset2['ganado']= 0
dataset2['counter']= 0

dataset_merge= dataset.append(dataset2).sort_values('Opportunity_Created_Date').reset_index().drop(columns=['index'])
#dataset_merge['counter'].value_counts()
dataset_merge.head()

/home/federico/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,ASP,ASP_(converted),ASP_(converted)_Currency,ASP_Currency,Account_Created_Date,Account_Name,Account_Owner,Account_Type,Billing_Country,Bureaucratic_Code,...,Source,Stage,TRF,Territory,Total_Amount,Total_Amount_Currency,Total_Taxable_Amount,Total_Taxable_Amount_Currency,counter,ganado
0,0.315,0.35629,USD,EUR,2013-07-27,Account_Name_34,Person_Name_16,Account_Type_5,France,Bureaucratic_Code_4,...,None,Closed Lost,2,France,662287.5,EUR,662287.5,EUR,1,0
1,0.490,0.49000,USD,USD,2014-01-22,Account_Name_272,Person_Name_4,Account_Type_5,Germany,Bureaucratic_Code_3,...,Source_3,Closed Lost,20,Philippines,9803430.0,USD,9803430.0,USD,1,0
2,0.680,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,None,Closed Lost,0,NW America,170931.6,USD,341863.2,USD,1,0
3,0.680,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,None,Closed Lost,0,NW America,170931.6,USD,341863.2,USD,1,0
4,0.740,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,Source_7,Closed Won,6,None,4395304.0,USD,31410780.0,USD,1,1


In [14]:
categorical = ['Region', 'Territory', 'Product_Name', 'Account_Owner','Bureaucratic_Code','Billing_Country', 
               'Account_Name', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms', 'Product_Family', 
               'Month', 'Delivery_Quarter']
for agrupador in categorical:
    gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
    gruped_product[agrupador + '_mean']= np.NaN
    gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
    gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })
    dataset_merge= dataset_merge.join(gruped_product)
    print(gruped_product.shape)

#agrupador = 'Opportunity_Owner' 
#gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
#gruped_product[agrupador + '_mean']= np.NaN
#gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
#gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })

#dataset_merge= dataset_merge.join(gruped_product)


(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)
(19434, 3)


In [15]:
dataset_merge.head()

,ASP,ASP_(converted),ASP_(converted)_Currency,ASP_Currency,Account_Created_Date,Account_Name,Account_Owner,Account_Type,Billing_Country,Bureaucratic_Code,...,Delivery_Terms_mean,Product_Family_ganados,Product_Family_count,Product_Family_mean,Month_ganados,Month_count,Month_mean,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean
0,0.315,0.35629,USD,EUR,2013-07-27,Account_Name_34,Person_Name_16,Account_Type_5,France,Bureaucratic_Code_4,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
1,0.490,0.49000,USD,USD,2014-01-22,Account_Name_272,Person_Name_4,Account_Type_5,Germany,Bureaucratic_Code_3,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
2,0.680,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
3,0.680,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,0.000000,0,2,0.0,0,1,0.0,0,2,0.0
4,0.740,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,0.333333,1,1,1.0,1,1,1.0,1,1,1.0


In [16]:
dataset.dtypes


ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

# categorical de a pares

In [17]:
for i in range(len(categorical) -1):
    for j in range(i +1, len(categorical)):
        gruped_product = dataset_merge.groupby([categorical[i],categorical[j]])[['ganado','counter']].cumsum()
        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean']= np.NaN
        gruped_product.loc[(gruped_product['counter'] != 0), categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product['ganado']/ gruped_product['counter']
        gruped_product= gruped_product.rename(columns = {'ganado':(categorical[i] + '_'+ categorical[j] + '_ganados'), 'counter': (categorical[i] + '_'+ categorical[j] + '_count') })
        dataset_merge= dataset_merge.join(gruped_product)
        

In [18]:
dataset.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

# Convertir mondeas

In [19]:
dataset_merge['conversion'] = dataset_merge['ASP_(converted)']/dataset_merge['ASP']

cotizacion = dataset_merge.groupby(['ASP_Currency', 'Opportunity_Created_Date']).agg({'conversion': 'mean'}).reset_index()
#print(dataset[dataset['Stage']== 'Closed Won']['conversion'].value_counts())
cotizacion = cotizacion.rename(columns = {'conversion': 'conversion_diaria'})
cotizacion

dataset_merge= dataset_merge.merge(cotizacion, on = ['ASP_Currency', 'Opportunity_Created_Date'], how = 'left')

dataset_merge

,ASP,ASP_(converted),ASP_(converted)_Currency,ASP_Currency,Account_Created_Date,Account_Name,Account_Owner,Account_Type,Billing_Country,Bureaucratic_Code,...,Product_Family_Month_count,Product_Family_Month_mean,Product_Family_Delivery_Quarter_ganados,Product_Family_Delivery_Quarter_count,Product_Family_Delivery_Quarter_mean,Month_Delivery_Quarter_ganados,Month_Delivery_Quarter_count,Month_Delivery_Quarter_mean,conversion,conversion_diaria
0,0.3150,0.35629,USD,EUR,2013-07-27,Account_Name_34,Person_Name_16,Account_Type_5,France,Bureaucratic_Code_4,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.131079,1.131079
1,0.4900,0.49000,USD,USD,2014-01-22,Account_Name_272,Person_Name_4,Account_Type_5,Germany,Bureaucratic_Code_3,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.000000,1.000000
2,0.6800,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.000000,1.000000
3,0.6800,0.68000,USD,USD,2013-08-22,Account_Name_359,Person_Name_45,Account_Type_2,United States,Bureaucratic_Code_5,...,1,0.000000,0,2,0.000000,0,1,0.000000,1.000000,1.000000
4,0.7400,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,1,1.000000,1,1,1.000000,1,1,1.000000,1.000000,1.000000
5,0.7400,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,1,1.000000,1,1,1.000000,2,2,1.000000,1.000000,1.000000
6,0.5500,0.55000,USD,USD,2013-07-28,Account_Name_1711,Person_Name_49,Account_Type_2,Pakistan,Bureaucratic_Code_5,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.000000,1.000000
7,0.7400,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,2,1.000000,2,2,1.000000,3,3,1.000000,1.000000,1.000000
8,0.7400,0.74000,USD,USD,2013-08-22,Account_Name_1337,Person_Name_38,Account_Type_2,United States,Bureaucratic_Code_5,...,1,1.000000,1,1,1.000000,4,4,1.000000,1.000000,1.000000
9,0.4200,0.42000,USD,USD,2014-07-11,Account_Name_1190,Person_Name_44,None,Philippines,Bureaucratic_Code_4,...,1,0.000000,0,2,0.000000,4,5,0.800000,1.000000,1.000000


In [20]:
dataset_merge['conversion'] = dataset_merge.loc[:,'conversion'].fillna(dataset_merge['conversion_diaria'])

In [21]:
dataset_merge = dataset_merge.drop(dataset_merge[dataset_merge['conversion'] == np.inf].index)

In [22]:
dataset_merge.loc[dataset_merge['ASP_Currency'] == 'USD', 'conversion'] = 1
dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY', 'conversion'].mean())
dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR', 'conversion'].mean())
dataset_merge['Total_Amount_converted'] = dataset_merge['Total_Amount'] * dataset_merge['conversion']

# diferencias de tiempo

In [23]:
dataset_merge['delta_planed'] = dataset_merge['Planned_Delivery_End_Date'] - dataset_merge['Planned_Delivery_Start_Date']

In [24]:
dataset_merge.dtypes

ASP                                                float64
ASP_(converted)                                    float64
ASP_(converted)_Currency                            object
ASP_Currency                                        object
Account_Created_Date                        datetime64[ns]
Account_Name                                        object
Account_Owner                                       object
Account_Type                                        object
Billing_Country                                     object
Bureaucratic_Code                                   object
Bureaucratic_Code_0_Approval                         int64
Bureaucratic_Code_0_Approved                         int64
Delivery_Quarter                                    object
Delivery_Terms                                      object
Delivery_Year                                        int64
ID                                                   int64
Last_Modified_By                                    obje

In [25]:
dataset_merge['acount_age'] = dataset_merge['Opportunity_Created_Date'] - dataset_merge['Account_Created_Date']

# features por oportunity ID

In [26]:
#Opportunity_ID
features= ['sum','mean','median','std', 'max', 'min']

dataset_numerico = dataset_merge[['Opportunity_ID','ASP', 'ASP_(converted)', 'TRF', 'Total_Amount', 'Total_Taxable_Amount', 'Total_Amount_converted']]
#ASP                                           float64
#ASP_(converted)                               float64
#TRF                                             int64
#Total_Amount                                  float64
#Total_Taxable_Amount                          float64
#Total_Amount_converted                        float64

features_num= dataset_numerico.groupby('Opportunity_ID').agg(features)
level_0 = features_num.columns.get_level_values(0)
level_1 = features_num.columns.get_level_values(1)
features_num.columns = level_0 + '_' + level_1
features_num.reset_index()
dataset_merge = dataset_merge.join(features_num, on= 'Opportunity_ID', how = 'left')

In [27]:
dataset.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

# elimino las columnas no propias

In [28]:
filter = dataset2['ID'].tolist()

In [29]:
dataset_merge.shape

(19433, 358)

In [30]:

dataset_merge = dataset_merge.drop(columns=['Region', 'Territory','Bureaucratic_Code','Account_Type', 'Billing_Country', 'Opportunity_Type',
                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
                            'Pricing, Delivery_Terms_Quote_Appr',
                            'Sales_Contract_No', 'Opportunity_ID', 'Opportunity_Name','Source ', 'Account_Created_Date',
                            'Bureaucratic_Code_0_Approval', 'Bureaucratic_Code_0_Approved','Pricing, Delivery_Terms_Approved',
               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter','ganado','counter',
                            'Total_Taxable_Amount_Currency','Total_Taxable_Amount','Total_Amount_Currency','Total_Amount',
                           'TRF','Delivery_Year','Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'ASP_(converted)_Currency',
                           'ASP_(converted)', 'ASP_Currency', 'ASP', 'Quote_Expiry_Date', 'Last_Modified_Date','Opportunity_Created_Date'])

dataset_merge.shape

(19433, 316)

In [31]:
Test = dataset_merge.loc[dataset_merge['ID'].isin(filter)]
print(Test.shape)
Train = dataset_merge.loc[~dataset_merge['ID'].isin(filter)]
print(Train.shape)          

(2551, 316)
(16882, 316)


In [32]:
Test.to_csv('Features_Test_Lucas.csv')
Train.to_csv('Features_Train_Lucas.csv')